In [ ]:
# Scraping the FlipKart Mobile Phones Data

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
final = pd.DataFrame()

for i in range(1,42):
    webpage = requests.get("https://www.flipkart.com/search?q=smartphones&page={}".format(i)).text
    soup = BeautifulSoup(webpage,'lxml')
    page = soup.find_all('div',class_="_2kHMtA")
    name = []
    link = []
    rating = []
    total_ratings = []
    total_reviews =[]
    discounted_price =[]
    actual_price = []
    for i in page:
#     Name of the image
        name.append(i.find('img',class_="_396cs4")['alt'])
#     Link of the image
        link.append(i.find('img',class_="_396cs4")['src'])
#     Rating
        rating.append(i.find('div',class_="_3LWZlK").text if i.find('div',class_="_3LWZlK") else "N/A")

#     Total Ratings
        
        total_ratings.append(i.find('span',class_="_2_R_DZ").find_all('span')[1].text.strip() if i.find('span',class_="_2_R_DZ") else "N/A")
#     Total Reviews
        total_reviews.append(i.find('span',class_="_2_R_DZ").find_all('span')[3].text.strip() if i.find('span',class_="_2_R_DZ") else "N/A")
#  Fetching the actuall price 
        actual_price.append(i.find('div',class_="_3I9_wc _27UcVY").text.strip() if i.find('div',class_="_3I9_wc _27UcVY") else "N/A")
# Fetching the dicsounted price 
        discounted_price.append(i.find('div',class_="_30jeq3 _1_WHN1").text.strip() if i.find('div',class_="_30jeq3 _1_WHN1") else "N/A")
    
#   Making the DataFrame and storing it in data 
    data = pd.DataFrame({'Name':name,'Link':link,'Rating':rating,
              'Total_Ratings':total_ratings,'Total_Reviews':total_reviews,
            'Actual_Price':actual_price,'Discounted_Price':discounted_price
             })
    final = pd.concat([final,data],ignore_index=True)
    # Make the loop stop for 3 sec so that scraping is not Blocked by robots 
    time.sleep(3)
#   Converting the pandas DataFrame to a csv file 
final.to_csv('Flipcart_mobile1.csv')

final

In [2]:
# Opening the File 
df = pd.read_csv("Flipcart_mobile1.csv")
df.head(4)

In [ ]:
df['Brand'] = df['Name'].str.split().str.get(0)
df['Name'] = df['Name'].str.split(' ',n=1).str.get(1)
df['Total_Ratings']=df['Total_Ratings'].str.split().str.get(0)
df['Total_Reviews'] = df['Total_Reviews'].str.split().str.get(0)
df['Actual_Price'] = df['Actual_Price'].str.split('₹').str.get(1)
df['Actual_Price'] = df['Actual_Price'].str.replace(",",'')
df['Discounted_Price']=df['Discounted_Price'].str.replace('[^\d.]','',regex=True)

df.set_index('Brand',inplace=True)

In [ ]:
df['Rating']=df['Rating'].fillna(0)
df['Total_Ratings']=df['Total_Ratings'].fillna(0)
df['Total_Reviews']=df['Total_Reviews'].fillna(0)
df['Actual_Price']=df['Actual_Price'].fillna(0)
df['Discounted_Price']=df['Discounted_Price'].fillna(0)


In [ ]:
df['Actual_Price']=df['Actual_Price'].astype(int)
df['Discounted_Price']=df['Discounted_Price'].astype(int)

In [ ]:
dis = abs(((df['Actual_Price'] - df['Discounted_Price'])*100)/df['Actual_Price']).round(0)
df['Discount %'] = dis.replace([np.inf,-np.inf],'No Discount')

In [ ]:
final = df[['Name','Link','Rating','Total_Ratings','Total_Reviews','Actual_Price','Discounted_Price','Discount %']]

In [ ]:
final